In [1]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np

model = VGG16(weights=None, include_top=True)
model.layers.pop()
for l in model.layers:
    if l.name == 'fc2':
        l.trainable = True
    else:
        l.trainable =False
        
from keras.layers import Dense
from keras.models import Model
from keras.optimizers import Adam

new_layer = Dense(2, activation='softmax', name='my_dense')

inp = model.input
out = new_layer(model.layers[-1].output)

model = Model(inp, out)
model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.001),metrics=['accuracy'])

model.summary()

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [2]:
model.load_weights('/home/ubuntu/git/learningWithKaggle/ft2_sample.h5')

In [3]:
layers = model.layers

In [4]:
from keras.layers import Convolution2D
last_conv_idx = [index for index,layer in enumerate(layers) 
                     if type(layer) is Convolution2D][-1]

layers[last_conv_idx]

In [5]:
from keras.models import Sequential

conv_layers = layers[:last_conv_idx+1]
conv_model = Sequential(conv_layers)
# Dense layers - also known as fully connected or 'FC' layers
fc_layers = layers[last_conv_idx+1:]

In [6]:
from keras.utils.np_utils import to_categorical  

def get_batches(dirname, gen=image.ImageDataGenerator(), shuffle=True, batch_size=4, class_mode='categorical',
                target_size=(224,224)):     
    return gen.flow_from_directory(dirname, target_size=target_size,
            class_mode=class_mode, shuffle=shuffle, batch_size=batch_size) 

def onehot(x):
    return to_categorical(x) 

In [7]:
path = '/home/ubuntu/data/dogscats/sample/'
batch_size = 16

batches = get_batches(path+'train', shuffle=False, batch_size=batch_size)
val_batches = get_batches(path+'valid', shuffle=False, batch_size=batch_size)

val_classes = val_batches.classes
trn_classes = batches.classes
val_labels = onehot(val_classes)
trn_labels = onehot(trn_classes)

Found 160 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


In [8]:
val_features = conv_model.predict_generator(val_batches, val_batches.n)


In [9]:
trn_features = conv_model.predict_generator(batches, batches.n)

In [10]:
import bcolz

def save_array(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()  

def load_array(fname):
    return bcolz.open(fname)[:]     

In [11]:
model_path = '/home/ubuntu/git/learningWithKaggle/' 
save_array(model_path + 'train_convlayer_features.bc', trn_features)
save_array(model_path + 'valid_convlayer_features.bc', val_features)
trn_features = load_array(model_path+'train_convlayer_features.bc')
val_features = load_array(model_path+'valid_convlayer_features.bc')

In [12]:
trn_features.shape

(2560, 14, 14, 512)

In [13]:
from keras.optimizers import RMSprop

# Such a finely tuned model needs to be updated very slowly!
opt = RMSprop(lr=0.0001, rho=0.7)

In [14]:
# Copy the weights from the pre-trained model.

def proc_wgts(layer): return [o for o in layer.get_weights()]

In [15]:
from keras.layers import MaxPooling2D, Dense, Flatten, Dropout

def get_fc_model():
    model = Sequential([
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dense(4096, activation='relu'),
        Dropout(0.2),
        Dense(4096, activation='relu'),
        Dropout(0.4),
        Dense(2, activation='softmax')
        ])

#     for l1,l2 in zip(model.layers, fc_layers): l1.set_weights(proc_wgts(l2))

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [16]:
fc_model = get_fc_model()
fc_layers = fc_model.layers

In [17]:
for l in fc_layers:
    print(l.name)
    

max_pooling2d_1
flatten_1
dense_1
dropout_1
dense_2
dropout_2
dense_3


In [18]:
conv_model.add(fc_model)
conv_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [19]:
conv_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [20]:
from keras.preprocessing.image import ImageDataGenerator

test_datagen = ImageDataGenerator(rescale=1./255)
train_datagen = ImageDataGenerator(rescale=1./255)
# train_datagen = ImageDataGenerator(
#         rescale=1./255,
#         shear_range=0.2,
#         zoom_range=0.2,
#         horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
        '/home/ubuntu/data/dogscats/sample/train',
        target_size=(224, 224),
        batch_size=8,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        '/home/ubuntu/data/dogscats/sample/valid',
        target_size=(224, 224),
        batch_size=8,
        class_mode='categorical')

conv_model.fit_generator(
        train_generator,
        steps_per_epoch=10,
        epochs=5,
        validation_data=validation_generator,
        validation_steps=40)

Found 160 images belonging to 2 classes.
Found 40 images belonging to 2 classes.
Epoch 1/5
10/10 [==============================] - 10s - loss: 2.6205 - acc: 0.5000 - val_loss: 2.5616 - val_acc: 0.5000
Epoch 2/5
10/10 [==============================] - 9s - loss: 1.4866 - acc: 0.6125 - val_loss: 0.9146 - val_acc: 0.5500
Epoch 3/5
10/10 [==============================] - 9s - loss: 1.3057 - acc: 0.6250 - val_loss: 1.5138 - val_acc: 0.5000
Epoch 4/5
10/10 [==============================] - 9s - loss: 0.7582 - acc: 0.7250 - val_loss: 2.8505 - val_acc: 0.5000
Epoch 5/5
10/10 [==============================] - 9s - loss: 0.8631 - acc: 0.6625 - val_loss: 2.6906 - val_acc: 0.5000


In [50]:
conv_model.save_weights(model_path + 'dropout0.h5')

In [21]:
conv_model.load_weights(model_path + 'dropout0.h5')

In [22]:
seq = conv_model.layers[-1]
drop = seq.layers[-4]
drop.rate

0.2

In [24]:
# need to change the dropout and reload the weights into the model, 
# now I have trained with dropout

from keras.preprocessing.image import ImageDataGenerator

test_datagen = ImageDataGenerator(rescale=1./255)
#train_datagen = ImageDataGenerator(rescale=1./255)
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
        '/home/ubuntu/data/dogscats/sample/train',
        target_size=(224, 224),
        batch_size=8,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        '/home/ubuntu/data/dogscats/sample/valid',
        target_size=(224, 224),
        batch_size=8,
        class_mode='categorical')

conv_model.fit_generator(
        train_generator,
        steps_per_epoch=160,
        epochs=5,
        validation_data=validation_generator,
        validation_steps=40)

Found 160 images belonging to 2 classes.
Found 40 images belonging to 2 classes.
Epoch 1/5
160/160 [==============================] - 49s - loss: 0.1601 - acc: 0.9805 - val_loss: 1.1409 - val_acc: 0.8500
Epoch 2/5
160/160 [==============================] - 49s - loss: 0.2713 - acc: 0.9750 - val_loss: 2.5668 - val_acc: 0.8000
Epoch 3/5
160/160 [==============================] - 49s - loss: 0.1625 - acc: 0.9820 - val_loss: 2.3821 - val_acc: 0.8250
Epoch 4/5
160/160 [==============================] - 49s - loss: 0.0914 - acc: 0.9906 - val_loss: 2.0200 - val_acc: 0.8250
Epoch 5/5
160/160 [==============================] - 49s - loss: 0.1148 - acc: 0.9906 - val_loss: 2.2101 - val_acc: 0.8500


In [25]:
seq = conv_model.layers[-1]
drop = seq.layers[-2]
drop.rate

0.4

In [26]:
seq.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 512)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 4096)              102764544 
_________________________________________________________________
dropout_1 (Dropout)          (None, 4096)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 4096)              16781312  
_________________________________________________________________
dropout_2 (Dropout)          (None, 4096)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 8194      
Total para

In [27]:
conv_layers[-1].output_shape[1:]

(14, 14, 512)

In [28]:
from keras.layers import BatchNormalization

def get_bn_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dense(4096, activation='relu'),
        Dropout(p),
        BatchNormalization(),
        Dense(4096, activation='relu'),
        Dropout(p),
        BatchNormalization(),
        Dense(2, activation='softmax')
        ]

In [29]:
p=0.6

In [30]:
bn_model = Sequential(get_bn_layers(0.6))

In [31]:
bn_model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 512)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 4096)              102764544 
_________________________________________________________________
dropout_3 (Dropout)          (None, 4096)              0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 4096)              16384     
_________________________________________________________________
dense_5 (Dense)              (None, 4096)              16781312  
_________________________________________________________________
dropout_4 (Dropout)          (None, 4096)              0         
__________

In [32]:
conv_layers = layers[:last_conv_idx+1]
conv_model2 = Sequential(conv_layers)

In [33]:
conv_model2.add(bn_model)

In [34]:
conv_model2.compile(Adam(), 'categorical_crossentropy', metrics=['accuracy'])

In [57]:
conv_model2.optimizer.lr = 0.001

In [56]:
from keras.preprocessing.image import ImageDataGenerator

test_datagen = ImageDataGenerator(rescale=1./255)
#train_datagen = ImageDataGenerator(rescale=1./255)
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
        '/home/ubuntu/data/dogscats/train',
        target_size=(224, 224),
        batch_size=8,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        '/home/ubuntu/data/dogscats/valid',
        target_size=(224, 224),
        batch_size=8,
        class_mode='categorical')

conv_model2.fit_generator(
        train_generator,
        steps_per_epoch=200,
        epochs=1,
        validation_data=validation_generator,
        validation_steps=200)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Epoch 1/1
200/200 [==============================] - 148s - loss: 0.4210 - acc: 0.8353 - val_loss: 0.2146 - val_acc: 0.9053


In [55]:
conv_model2.optimizer.lr = 0.0001

In [53]:
conv_model2.save_weights('/home/ubuntu/git/learningWithKaggle/ft2_batchNorm.h5')

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

test_datagen = ImageDataGenerator(rescale=1./255)
#train_datagen = ImageDataGenerator(rescale=1./255)
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
        '/home/ubuntu/data/dogscats/train',
        target_size=(224, 224),
        batch_size=16,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        '/home/ubuntu/data/dogscats/valid',
        target_size=(224, 224),
        batch_size=16,
        class_mode='categorical')

conv_model2.fit_generator(
        train_generator,
        steps_per_epoch=200,
        epochs=10,
        validation_data=validation_generator,
        validation_steps=200)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Epoch 1/10
200/200 [==============================] - 148s - loss: 0.4177 - acc: 0.8303 - val_loss: 0.2198 - val_acc: 0.9034
Epoch 2/10
200/200 [==============================] - 148s - loss: 0.4107 - acc: 0.8291 - val_loss: 0.2522 - val_acc: 0.8859
Epoch 3/10
200/200 [==============================] - 148s - loss: 0.4097 - acc: 0.8344 - val_loss: 0.1932 - val_acc: 0.9216
Epoch 4/10
199/200 [============================>.] - ETA: 0s - loss: 0.4044 - acc: 0.8320